In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [2]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [3]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

item_cnt_day item_price   revenue
                                        sum     median       sum
date_block_num shop_id item_id                                  
21             15      12977            1.0    199.000    199.00
3              56      2718             2.0    299.000    598.00
               53      9971             1.0    149.000    149.00
26             51      3708             2.0    623.315   1246.63
7              25      19404            5.0   1799.000   8994.50
4              52      14465            1.0    299.000    299.00
2              50      13425            3.0  12490.000  37470.00
22             42      15023            2.0    104.150    208.30
21             55      4859             3.0   1750.000   5250.00
17             57      12926            1.0    399.000    399.00

In [4]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [5]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [6]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
4474263,135583,48,12523,25,NaN,NaN,NaN,55
32235,976,5,4564,28,NaN,NaN,NaN,55
6792536,205834,39,13743,15,NaN,NaN,NaN,70
3305054,100153,19,13316,6,NaN,NaN,NaN,47
1915970,58059,25,20403,24,NaN,NaN,NaN,72
2363359,71616,21,6476,32,1.0,249.0,249.0,28
5559302,168463,37,10659,24,NaN,NaN,NaN,67
2155931,65331,22,18069,9,NaN,NaN,NaN,62
3154293,95584,14,19334,22,NaN,NaN,NaN,49
5037334,152646,59,19539,17,NaN,NaN,NaN,40


In [7]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
3632724  110082       50    12513              19          0.0           0.0   
5613014  170091       37    11913              12          0.0           0.0   
5247002  159000       56     1239               3          0.0           0.0   
3148409   95406       14    10391              12          0.0           0.0   
3520293  106675       42     2004              19          0.0           0.0   
5338420  161770       56    14361              11          0.0           0.0   
5527866  167511       36     9530               4          0.0           0.0   
3643561  110410       50    12332              32          0.0           0.0   
6915983  209575       45    22002               9          0.0           0.0   
1989766   60295       25    16232              32          1.0         599.0   

         total_revenue  item_category_id  
3632724            0.0                55  
5613014            0.0           

In [8]:
whole_df.sort_values(by=['ID', 'date_block_num'])
whole_df = whole_df[whole_df['item_id'] <= 5000]
whole_df.drop(['ID', 'total_revenue'], axis=1, inplace=True)
whole_df = whole_df[['total_count', 'date_block_num', 'shop_id', 'item_id', 'item_category_id', 'median_price']]
print(whole_df.head(34))
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(49644, 33, 6)
print(whole_df_arr)#change back to 214200

     total_count  date_block_num  shop_id  item_id  item_category_id  \
330          0.0               1        5     4806                30   
331          0.0               2        5     4806                30   
332          0.0               3        5     4806                30   
333          0.0               4        5     4806                30   
334          0.0               5        5     4806                30   
335          0.0               6        5     4806                30   
336          0.0               7        5     4806                30   
337          0.0               8        5     4806                30   
338          0.0               9        5     4806                30   
339          0.0              10        5     4806                30   
340          0.0              11        5     4806                30   
341          1.0              12        5     4806                30   
342          2.0              13        5     4806              

In [9]:
print(whole_df_arr)

[[[0.000e+00 1.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  [0.000e+00 2.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  [0.000e+00 3.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  ...
  [5.000e+00 3.100e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]
  [2.000e+00 3.200e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]
  [3.000e+00 3.300e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]]

 [[0.000e+00 1.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 2.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 3.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  ...
  [0.000e+00 3.100e+01 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [1.000e+00 3.200e+01 5.000e+00 4.843e+03 2.000e+01 9.999e+03]
  [0.000e+00 3.300e+01 5.000e+00 4.843e+03 2.000e+01 0.000e+00]]

 [[0.000e+00 1.000e+00 5.000e+00 4.607e+03 5.500e+01 0.000e+00]
  [1.000e+00 2.000e+00 5.000e+00 4.607e+03 5.500e+01 2.990e+02]
  [0.000e+00 3.000e+00 5.000e+00 4.607e+03 5.500e+01 0.000e+00]
  ...
  [0.000e+00 3.100

In [10]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)
train_arr_first_test = train_arr[:, 19:25]
print(train_arr_first_test.shape)
print(train_arr)

(49644, 26, 6)
(49644, 7, 6)
(49644, 6, 6)
[[[0.000e+00 1.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  [0.000e+00 2.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  [0.000e+00 3.000e+00 5.000e+00 4.806e+03 3.000e+01 0.000e+00]
  ...
  [4.000e+00 2.400e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]
  [3.000e+00 2.500e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]
  [6.000e+00 2.600e+01 5.000e+00 4.806e+03 3.000e+01 9.900e+01]]

 [[0.000e+00 1.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 2.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 3.000e+00 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  ...
  [0.000e+00 2.400e+01 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 2.500e+01 5.000e+00 4.843e+03 2.000e+01 0.000e+00]
  [0.000e+00 2.600e+01 5.000e+00 4.843e+03 2.000e+01 0.000e+00]]

 [[0.000e+00 1.000e+00 5.000e+00 4.607e+03 5.500e+01 0.000e+00]
  [1.000e+00 2.000e+00 5.000e+00 4.607e+03 5.500e+01 2.990e+02]
  [0.000e+00 3.000e+00 5.000e+00 4.607e+03 5.

In [11]:
#normalise data

train_mean = train_arr.mean(axis=(0, 1))
train_std = train_arr.std(axis=(0, 1))
train_arr = (train_arr - train_mean)/train_std
print(train_arr)
val_arr = (val_arr - train_mean)/train_std


[[[-1.22549950e-01 -1.66666667e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01 -2.31336187e-01]
  [-1.22549950e-01 -1.53333333e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01 -2.31336187e-01]
  [-1.22549950e-01 -1.40000000e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01 -2.31336187e-01]
  ...
  [ 1.53205032e+00  1.40000000e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01  8.50524087e-04]
  [ 1.11840025e+00  1.53333333e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01  8.50524087e-04]
  [ 2.35935045e+00  1.66666667e+00 -1.51708349e+00  1.51896332e+00
   -6.13688286e-01  8.50524087e-04]]

 [[-1.22549950e-01 -1.66666667e+00 -1.51708349e+00  1.54566912e+00
   -1.14304649e+00 -2.31336187e-01]
  [-1.22549950e-01 -1.53333333e+00 -1.51708349e+00  1.54566912e+00
   -1.14304649e+00 -2.31336187e-01]
  [-1.22549950e-01 -1.40000000e+00 -1.51708349e+00  1.54566912e+00
   -1.14304649e+00 -2.31336187e-01]
  ...
  [-1.22549950e-01  1.40000000e+00 -1.51708349e+00  1.54566

In [12]:
#train_arr = list(train_arr)

#train_arr = train_arr[0:3]
train_list = list(train_arr)
val_list = list(val_arr)
first_test_list = list(train_arr_first_test)
#def split_sequences(sequences, n_steps):
#    pass


In [ ]:
def windowed_dataset(array, window_size):
    ds = tf.data.Dataset.from_tensor_slices(array)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    ds = ds.map(lambda x: (x[:-1], x[-1][0]))
    return ds
#dataset = windowed_dataset(train_arr, window_size, batch_size)
#for d in dataset:
#   print(d)

In [14]:
print(train_arr.shape)
dataset = tf.data.Dataset.from_generator(lambda: train_list, tf.as_dtype(train_list[0].dtype), tf.TensorShape([26,6]))
dataset = dataset.flat_map(lambda x: windowed_dataset(x, window_size))
dataset = dataset.batch(2000).prefetch(1)

(49644, 26, 6)


In [15]:
val_dataset = tf.data.Dataset.from_generator(lambda: val_list, tf.as_dtype(val_list[0].dtype), tf.TensorShape([7,6]))
val_dataset = val_dataset.flat_map(lambda x: windowed_dataset(x, window_size))
val_dataset = val_dataset.batch(2000).prefetch(1)

In [16]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1,  input_shape=(6,6))
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(dataset,validation_data=val_dataset, epochs=100,verbose=1)

Epoch 1/100
497/497 [==============================] - 97s 195ms/step - loss: 2.5798 - val_loss: 1.4697

KeyboardInterrupt: 

In [ ]:
#print(val_arr)
#print(train_arr[:,-7:-1])
#create prediction window excluding last value to compare
predictions = model.predict(val_arr[:,:-1])
predictions = model.predict(train_arr[:,-7:-1])
denormalised_predictions = predictions*train_std[0] + train_mean[0]

In [ ]:

print(denormalised_predictions.astype(int))
print(train_mean)
print(train_std)